In [1]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import io
from scipy import ndimage
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings')

sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2131064230076894398
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1508248780
locality {
  bus_id: 1
  links {
  }
}
incarnation: 482233857291846942
physical_device_desc: "device: 0, name: GeForce GT 740M, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


In [2]:
def plot_advections(advected_aux_gen, advected_aux_truth):
    gen = advected_aux_gen[:5]
    truth =  advected_aux_truth[:5]
    fig, axs = plt.subplots(5, 2, figsize=(16, 16))
    for i in range(5):
        vmax = np.max([np.max(gen[i]), np.max(truth[i])])
        vmin = 0
        im = axs[i,0].imshow(gen[i, :,:,0], vmax=vmax,vmin=vmin)
        axs[i,0].axis('off')
        src.colorbar(im)
        axs[i,0].set_title("Advected generated frame")
        
        im = axs[i,1].imshow(truth[i, :,:,0], vmax=vmax,vmin=vmin)
        axs[i,1].axis('off')
        src.colorbar(im)
        axs[i,1].set_title("Advected reference frame")
        
    fig.savefig("Plots/advections_epoch %d.png" % epoch)
    plt.close()

In [3]:
past = 2
name = f"tgan_{past}-1"

Load dataset.

In [4]:
train, xval, test = src.load_datasets(past_frames=past+1)

Training data: (7500, 64, 64, 4)
Validation data: (1500, 64, 64, 4)
Test data: (1000, 64, 64, 4)


Split data to inputs and ground truth images.

In [5]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past+1, augment=True)

Data augmentation.
Shape of training data:  (8000, 64, 64, 3) 
Shape of training truth:  (8000, 64, 64, 1) 
Shape of validation data:  (1500, 64, 64, 3) 
Shape of validation truth:  (1500, 64, 64, 1) 
Shape of test data:  (1000, 64, 64, 3) 
Shape of test truth:  (1000, 64, 64, 1)


Calculate optical flows between frame t-1 and t.

In [ ]:
#optical flow of the augmented data of the first 2000 training images (8000 images)
#vx, vy = src.optical_flow(gan_train[:,:,:,-2:-1], gan_train[:,:,:,-1:], window_size=4, tau=1e-2, init=1) # (n,:,:,1)

Save optical flows

In [ ]:
#np.savez_compressed(f"{name}_vx_2000",vx) #2000 denotes that they re the flo2 of the first 2000 samples from the training dataset
#np.savez_compressed(f"{name}_vy_2000",vy)

If optical flows are saved, load them

In [6]:
vx = np.load(sys.path[0]+f"/{name}_vx_2000.npz")["arr_0"]
vy = np.load(sys.path[0]+f"/{name}_vy_2000.npz")["arr_0"]

Preprocess optical flows

In [7]:
normalized_optical_flow = src.normalize_flows(vx, vy)


In [8]:
flows = np.transpose([[ndimage.median_filter(image[..., ch], 4) for ch in range(2)] for image in normalized_optical_flow], (0, 2, 3, 1))

Make discriminator labels.

In [9]:
batch_size=16

In [10]:
# Adversarial ground truths
real = -np.ones((batch_size, 1))
fake = np.ones((batch_size, 1))

Switch 5% of labels and smooth from 1 to 0.9.

In [11]:
#real, fake = src.noisy_d_labels(real, fake)

Make generator but don't compile.

In [12]:
generator = src.unet((64, 64, past), dropout=0, batchnorm=True, kernel_size=4, feature_mult=1)

Make discriminators and compile.

In [13]:
s_discriminator = src.spatial_discriminator(condition_shape=(64, 64, past), dropout = 0.25, batchnorm=True, wgan=True)
s_discriminator.compile(loss=src.wasserstein_loss,
                        optimizer=keras.optimizers.RMSprop(lr=0.00005))

In [14]:
t_discriminator = src.temporal_discriminator(dropout = 0.25, batchnorm=True, wgan=True)
t_discriminator.compile(loss=src.wasserstein_loss,
                        optimizer=keras.optimizers.RMSprop(lr=0.00005))

Inputs and outputs of the GAN.

In [15]:
frame_t = keras.layers.Input(shape=(64, 64, past))
adv = keras.layers.Input(shape=(64, 64, 1))

In [16]:
generated = generator(frame_t)
s_score = s_discriminator([frame_t, generated])
t_score = t_discriminator([adv, generated])

Freeze discriminator weights.

In [17]:
s_discriminator.trainable = False
t_discriminator.trainable = False

Compile combined model.

In [18]:
loss_weights=[0, 1, 1]

In [19]:
combined = keras.models.Model(inputs=[frame_t, adv], outputs=[generated, s_score, t_score])

In [20]:
combined.compile(loss=[src.custom_loss(loss="l1"),
                       src.wasserstein_loss,
                       src.wasserstein_loss],
                 optimizer=keras.optimizers.RMSprop(lr=0.00005),
                 loss_weights=loss_weights)

***Using L1 loss.***


Train x epochs.

In [21]:
RND = 777
np.random.seed(RND)
iterations = 5000
g = 1
nb_batches = int(gan_train.shape[0] / batch_size)

In [22]:
log = {"g_loss":[],
       "d_loss":[],
       "ds_loss":[],
       "dt_loss":[],
       "g_metric":[],
       "d_metric":[],
       "ds_metric":[],
       "dt_metric":[],
       "d_loss_real":[],
       "d_loss_fake":[],
       "ds_loss_real":[],
       "ds_loss_fake":[],
       "dt_loss_real":[],
       "dt_loss_fake":[],}

In [ ]:
for it in range(iterations):
#batch
#idx = range(it%nb_batches * batch_size,(it%nb_batches + 1) * batch_size)
#DS create training batch
    if (it % 1000) < 25 or it % 500 == 0: # 25 times in 1000, every 500th
        d_iters = 100
    else:
        d_iters = 10
        
    for d_it in range(d_iters):
        s_discriminator.trainable = True
        for l in s_discriminator.layers: l.trainable = True
        idx = np.random.choice(gan_train.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        
        for l in s_discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
        
        generated_imgs = generator.predict(training_batch) 
        ds_loss_real = s_discriminator.train_on_batch([training_batch, real_imgs], real)
        ds_loss_fake = s_discriminator.train_on_batch([training_batch, generated_imgs], fake)

        ds_loss = 0.5 * np.add(ds_loss_real, ds_loss_fake)
        s_discriminator.trainable = False
        for l in s_discriminator.layers: l.trainable = False
    
#DT create training batch
        t_discriminator.trainable = True
        for l in t_discriminator.layers: l.trainable = True
        idx = np.random.choice(gan_train.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        aux_batch = gan_train[idx,:,:,:-1]
        
        for l in t_discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
            
        advected_aux_gen = generator.predict(aux_batch)
        for i in range(10):
            advected_aux_gen = np.array([src.advect(sample) for sample in np.concatenate((advected_aux_gen, -flows[idx]), axis=-1)])
        advected_aux_truth = training_batch[:,:,:,-1:]
        for i in range(10):
            advected_aux_truth = np.array([src.advect(sample) for sample in np.concatenate((advected_aux_truth, -flows[idx]), axis=-1)])
        
        generated_imgs = generator.predict(training_batch) 
        dt_loss_real = t_discriminator.train_on_batch([advected_aux_truth, real_imgs], real)
        dt_loss_fake = t_discriminator.train_on_batch([advected_aux_gen, generated_imgs], fake)

        dt_loss = 0.5 * np.add(dt_loss_real, dt_loss_fake)
        t_discriminator.trainable = False
        for l in t_discriminator.layers: l.trainable = False

#Total discriminator loss
        d_loss = 0.5 * np.add(ds_loss, dt_loss)
        d_loss_real = 0.5 * np.add(ds_loss_real, dt_loss_real)
        d_loss_fake = 0.5 * np.add(ds_loss_fake, dt_loss_fake)
        
        print(f"\033[1m{it+1}/{d_it} [D loss real: {d_loss_real} D loss fake: {d_loss_fake}]\033[0m \n"+
              f"{it}/{d_it} [Ds loss real: {ds_loss_real} Ds loss fake: {ds_loss_fake}] \n"+
              f"{it}/{d_it} [Dt loss real: {dt_loss_real} Dt loss fake: {dt_loss_fake}]")

#Generator
    for tg in range(g):
        idx = np.random.choice(gan_train.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        aux_batch = gan_train[idx,:,:,:-1]

        advected = generator.predict(aux_batch)
        for i in range(10):
            advected = np.array([src.advect(sample) for sample in np.concatenate((advected, -flows[idx]), axis=-1)])  
    
        g_loss = combined.train_on_batch([training_batch, advected], [real_imgs, real, real])
    
    log["g_loss"].append(g_loss)
    log["d_loss"].append(d_loss)
    log["ds_loss"].append(ds_loss) 
    log["dt_loss"].append(dt_loss)
    log["d_loss_real"].append(d_loss_real)
    log["d_loss_fake"].append(d_loss_fake)
    log["ds_loss_real"].append(ds_loss_real)
    log["ds_loss_fake"].append(ds_loss_fake)
    log["dt_loss_real"].append(dt_loss_real)
    log["dt_loss_fake"].append(dt_loss_fake)
    
    
    print(f"\033[1m {it} [Ds loss: {ds_loss}, Dt loss: {dt_loss}]\033[0m \n"+
          f"\033[1m {it} [G loss: {g_loss}]\033[0m \n"+
          f" Ds: real loss: {ds_loss_real}, fake loss: {ds_loss_fake} \n"+
          f" Dt: real loss: {dt_loss_real}, fake loss: {dt_loss_fake}")
    if it%100 == 0 and it>0:
        src.sample_images(it, gan_test[...,1:], gan_test_truth, past, generator)
        src.plot_temporal_training_curves(log, it, name, wgan=True)
        src.update_output("")

src.sample_images(iterations, gan_test[...,1:], gan_test_truth, past, generator)
src.plot_temporal_training_curves(log, iterations, name, wgan=True)


1802/0 [D loss real: -0.0023262295871973038 D loss fake: -0.004092153627425432] 
1802/0 [Ds loss real: -0.0005256922449916601 Ds loss fake: 0.0006788158789277077] 
1802/0 [Dt loss real: -0.004126766696572304 Dt loss fake: -0.008863123133778572]
1802/1 [D loss real: 0.00041322113247588277 D loss fake: -0.006039920262992382] 
1802/1 [Ds loss real: -0.0016989227151498199 Ds loss fake: 0.0004648128233384341] 
1802/1 [Dt loss real: 0.0025253649801015854 Dt loss fake: -0.012544653378427029]
1802/2 [D loss real: -0.0041893296875059605 D loss fake: -0.0013660159893333912] 
1802/2 [Ds loss real: -0.0014897293876856565 Ds loss fake: 0.0006630279822275043] 
1802/2 [Dt loss real: -0.006888930220156908 Dt loss fake: -0.0033950600773096085]
1802/3 [D loss real: -0.002506224438548088 D loss fake: -0.0028097499161958694] 
1802/3 [Ds loss real: -0.0013744526077061892 Ds loss fake: 0.00037759565748274326] 
1802/3 [Dt loss real: -0.0036379960365593433 Dt loss fake: -0.0059970952570438385]
1802/4 [D loss

1805/0 [D loss real: -0.00629589706659317 D loss fake: 0.0024052669759839773] 
1805/0 [Ds loss real: -0.0021642385981976986 Ds loss fake: 0.000951720925513655] 
1805/0 [Dt loss real: -0.010427555069327354 Dt loss fake: 0.0038588130846619606]
1805/1 [D loss real: -0.00069188023917377 D loss fake: -0.006655689794570208] 
1805/1 [Ds loss real: -0.0017613060772418976 Ds loss fake: 0.0002431102911941707] 
1805/1 [Dt loss real: 0.0003775455988943577 Dt loss fake: -0.01355449017137289]
1805/2 [D loss real: 0.005233972333371639 D loss fake: -0.011005649343132973] 
1805/2 [Ds loss real: -0.0019090104615315795 Ds loss fake: 0.0010550108272582293] 
1805/2 [Dt loss real: 0.012376954779028893 Dt loss fake: -0.023066310212016106]
1805/3 [D loss real: -0.007121762726455927 D loss fake: 0.004223508760333061] 
1805/3 [Ds loss real: -0.0002749554405454546 Ds loss fake: 0.001151079311966896] 
1805/3 [Dt loss real: -0.013968570157885551 Dt loss fake: 0.007295937743037939]
1805/4 [D loss real: -0.008524757

1808/0 [D loss real: -0.0027613539714366198 D loss fake: -0.0020049589220434427] 
1808/0 [Ds loss real: -0.0025213761255145073 Ds loss fake: 0.0005012316396459937] 
1808/0 [Dt loss real: -0.0030013318173587322 Dt loss fake: -0.004511149600148201]
1808/1 [D loss real: -0.004988042637705803 D loss fake: -0.0036741592921316624] 
1808/1 [Ds loss real: -0.0016494975425302982 Ds loss fake: 0.0011351406574249268] 
1808/1 [Dt loss real: -0.008326588198542595 Dt loss fake: -0.008483459241688251]
1808/2 [D loss real: -0.0014739225152879953 D loss fake: -0.006137885618954897] 
1808/2 [Ds loss real: -0.0023602917790412903 Ds loss fake: 0.0008510004263371229] 
1808/2 [Dt loss real: -0.0005875532515347004 Dt loss fake: -0.01312677189707756]
1808/3 [D loss real: -0.007826851680874825 D loss fake: -0.00014453439507633448] 
1808/3 [Ds loss real: -0.0018223439110442996 Ds loss fake: 0.0009419980924576521] 
1808/3 [Dt loss real: -0.013831359334290028 Dt loss fake: -0.001231066882610321]
1808/4 [D loss re

1811/0 [D loss real: -0.0038070569280534983 D loss fake: -0.0016861159820109606] 
1811/0 [Ds loss real: -0.001862207893282175 Ds loss fake: -0.00031646795105189085] 
1811/0 [Dt loss real: -0.0057519059628248215 Dt loss fake: -0.003055764129385352]
1811/1 [D loss real: -0.007409697398543358 D loss fake: 0.0008183705504052341] 
1811/1 [Ds loss real: -0.0015019276179373264 Ds loss fake: 0.00042803771793842316] 
1811/1 [Dt loss real: -0.013317467644810677 Dt loss fake: 0.001208703382872045]
1811/2 [D loss real: -0.003356355242431164 D loss fake: -0.0014431311283260584] 
1811/2 [Ds loss real: -0.0010913845617324114 Ds loss fake: 0.0004927361733280122] 
1811/2 [Dt loss real: -0.00562132615596056 Dt loss fake: -0.00337899848818779]
1811/3 [D loss real: -0.008557702414691448 D loss fake: 1.3794386177323759e-05] 
1811/3 [Ds loss real: -0.0017137874383479357 Ds loss fake: -0.00011732027633115649] 
1811/3 [Dt loss real: -0.01540161669254303 Dt loss fake: 0.000144909048685804]
1811/4 [D loss real:

1814/0 [D loss real: -0.003833989379927516 D loss fake: -0.002919405698776245] 
1814/0 [Ds loss real: -0.002198751550167799 Ds loss fake: 0.00053812051191926] 
1814/0 [Dt loss real: -0.005469227209687233 Dt loss fake: -0.00637693190947175]
1814/1 [D loss real: -0.008181178011000156 D loss fake: 0.002574643585830927] 
1814/1 [Ds loss real: -0.0005531754577532411 Ds loss fake: 0.00038831745041534305] 
1814/1 [Dt loss real: -0.015809180215001106 Dt loss fake: 0.0047609698958694935]
1814/2 [D loss real: -0.005566244944930077 D loss fake: -0.0006295557832345366] 
1814/2 [Ds loss real: -0.0018080949084833264 Ds loss fake: 0.00038646283792331815] 
1814/2 [Dt loss real: -0.009324395097792149 Dt loss fake: -0.0016455743461847305]
1814/3 [D loss real: -0.0035864212550222874 D loss fake: -0.002504670526832342] 
1814/3 [Ds loss real: -0.0014956013765186071 Ds loss fake: 0.0006574325961992145] 
1814/3 [Dt loss real: -0.005677241366356611 Dt loss fake: -0.005666773766279221]
1814/4 [D loss real: -0.

...

1866/4 [D loss real: -0.006849989760667086 D loss fake: -0.0028294073417782784] 
1866/4 [Ds loss real: -0.002201365539804101 Ds loss fake: -0.0003628377744462341] 
1866/4 [Dt loss real: -0.011498614214360714 Dt loss fake: -0.005295976996421814]
1866/5 [D loss real: -0.005121749825775623 D loss fake: 9.092400432564318e-06] 
1866/5 [Ds loss real: -0.0018670044373720884 Ds loss fake: -6.550803664140403e-05] 
1866/5 [Dt loss real: -0.008376494981348515 Dt loss fake: 8.369283750653267e-05]
1866/6 [D loss real: -0.006152448710054159 D loss fake: 0.00035416107857599854] 
1866/6 [Ds loss real: -0.001031451509334147 Ds loss fake: -0.00031354569364339113] 
1866/6 [Dt loss real: -0.011273445561528206 Dt loss fake: 0.0010218678507953882]
1866/7 [D loss real: -0.0005849614972248673 D loss fake: -0.006873150821775198] 
1866/7 [Ds loss real: -0.0010648774914443493 Ds loss fake: -0.00012765137944370508] 
1866/7 [Dt loss real: -0.0001050455030053854 Dt loss fake: -0.013618649914860725]
1866/8 [D loss r

1869/3 [D loss real: -0.010922843590378761 D loss fake: 0.0038681072182953358] 
1869/3 [Ds loss real: -0.0014750331174582243 Ds loss fake: -0.0015471300575882196] 
1869/3 [Dt loss real: -0.02037065476179123 Dt loss fake: 0.009283344261348248]
1869/4 [D loss real: -0.009502063505351543 D loss fake: 0.0009013981325551867] 
1869/4 [Ds loss real: -0.0009665052639320493 Ds loss fake: -0.0011099216062575579] 
1869/4 [Dt loss real: -0.018037620931863785 Dt loss fake: 0.0029127178713679314]
1869/5 [D loss real: -0.0006940363673493266 D loss fake: -0.0058672004379332066] 
1869/5 [Ds loss real: -0.0019018652383238077 Ds loss fake: -0.0014018930960446596] 
1869/5 [Dt loss real: 0.0005137925036251545 Dt loss fake: -0.010332508012652397]
1869/6 [D loss real: -0.008050577715039253 D loss fake: 0.0019367486238479614] 
1869/6 [Ds loss real: -0.0009776699589565396 Ds loss fake: -0.0012976963771507144] 
1869/6 [Dt loss real: -0.015123485587537289 Dt loss fake: 0.005171193741261959]
1869/7 [D loss real: 

1872/2 [D loss real: -0.0060571529902517796 D loss fake: -0.0008503473363816738] 
1872/2 [Ds loss real: -0.001832433044910431 Ds loss fake: -0.0010502354707568884] 
1872/2 [Dt loss real: -0.010281872935593128 Dt loss fake: -0.0006504592020064592]
1872/3 [D loss real: -0.010270479135215282 D loss fake: 0.0032634586095809937] 
1872/3 [Ds loss real: -0.0013459052424877882 Ds loss fake: -0.0002935522934421897] 
1872/3 [Dt loss real: -0.019195053726434708 Dt loss fake: 0.006820469629019499]
1872/4 [D loss real: -0.00605932017788291 D loss fake: -0.003321882104501128] 
1872/4 [Ds loss real: -0.0015837522223591805 Ds loss fake: -0.0015662238001823425] 
1872/4 [Dt loss real: -0.010534888133406639 Dt loss fake: -0.005077540408819914]
1872/5 [D loss real: -0.009295886382460594 D loss fake: -0.00017518035019747913] 
1872/5 [Ds loss real: -0.0012890421785414219 Ds loss fake: -0.0005529675981961191] 
1872/5 [Dt loss real: -0.017302731052041054 Dt loss fake: 0.0002026068978011608]
1872/6 [D loss rea

1875/1 [D loss real: -0.007038078736513853 D loss fake: -0.0002952786162495613] 
1875/1 [Ds loss real: -0.0011005689157173038 Ds loss fake: -0.0009748443262651563] 
1875/1 [Dt loss real: -0.01297558844089508 Dt loss fake: 0.00038428709376603365]
1875/2 [D loss real: -0.008478035219013691 D loss fake: 0.0006852247752249241] 
1875/2 [Ds loss real: -0.0012454784009605646 Ds loss fake: -0.0005210501258261502] 
1875/2 [Dt loss real: -0.01571059226989746 Dt loss fake: 0.0018914996180683374]
1875/3 [D loss real: -0.0037689926102757454 D loss fake: -0.0017767229583114386] 
1875/3 [Ds loss real: -0.0011624989565461874 Ds loss fake: -0.00026004266692325473] 
1875/3 [Dt loss real: -0.00637548603117466 Dt loss fake: -0.0032934031914919615]
1875/4 [D loss real: -0.007161816116422415 D loss fake: -0.002960111480206251] 
1875/4 [Ds loss real: -0.0022259086836129427 Ds loss fake: -0.0004197372472845018] 
1875/4 [Dt loss real: -0.01209772378206253 Dt loss fake: -0.00550048565492034]
1875/5 [D loss real

1878/0 [D loss real: -0.007923823781311512 D loss fake: -0.00011630053631961346] 
1878/0 [Ds loss real: 0.0002736373571678996 Ds loss fake: -0.002179163508117199] 
1878/0 [Dt loss real: -0.016121285036206245 Dt loss fake: 0.001946562435477972]
1878/1 [D loss real: -0.0045939283445477486 D loss fake: -0.0021580925676971674] 
1878/1 [Ds loss real: 0.00020492917974479496 Ds loss fake: -0.002320343628525734] 
1878/1 [Dt loss real: -0.009392785839736462 Dt loss fake: -0.001995841506868601]
1878/2 [D loss real: -0.0022426159121096134 D loss fake: -0.005363165400922298] 
1878/2 [Ds loss real: -0.0010735827963799238 Ds loss fake: -0.0019987502600997686] 
1878/2 [Dt loss real: -0.0034116487950086594 Dt loss fake: -0.008727580308914185]
1878/3 [D loss real: -0.004508621990680695 D loss fake: -0.0037272279150784016] 
1878/3 [Ds loss real: -0.0018886011093854904 Ds loss fake: -0.0014268008526414633] 
1878/3 [Dt loss real: -0.007128643337637186 Dt loss fake: -0.006027654744684696]
1878/4 [D loss re

In [ ]:
src.plot_temporal_training_curves(log, iterations, name, wgan=True)

In [ ]:
total_g_loss = np.array(log["g_loss"])[:,0]
total_d_loss = np.array(log["d_loss"])[:,0]
smoothed_tgl = src.smooth(np.array(log["g_loss"])[:,0])
smoothed_tdl = src.smooth(np.array(log["d_loss"])[:,0])
objective_loss = np.array(log["g_loss"])[:,1]

# plot 'em
f, (a0, a1) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[5, 2]})
a0.plot(total_g_loss, alpha=0.3, c="b")
a0.plot(total_d_loss, alpha=0.3, c="orange")
a0.plot(smoothed_tgl, c="b", label="generator")
a0.grid()
a0.plot(smoothed_tdl, c="orange", label="discriminator")
a0.legend()
a1.plot(objective_loss, alpha=0.9, c="green", label="L1 objective")
a1.grid()
a1.legend()
f.text(0.5, 0, 'Iterations', ha='center', va='center')
f.text(0, 0.5, 'Loss', ha='center', va='center', rotation='vertical')

f.tight_layout()
f.savefig(name+'_curves.png')

## Save features

Save model history

In [ ]:
np.save(name+"_log",log)

Save model weights

In [ ]:
combined.save_weights(name+"_model.h5")

Load model and predict

In [ ]:
combined.load_weights(sys.path[1]+"/"+name+"/"+name+"_model.h5")

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [ ]:
sequence_test = src.load_datasets(prediction=True)

In [ ]:
sequence_test = src.augment_data(sequence_test[:100])

In [ ]:
sequence_test.shape

In [ ]:
generator = combined.layers[1]

In [ ]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]
for t in range(5):
    future = generator.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

Save example predictions

In [ ]:
def save_examples(name, test, predictions_dict, past, samples=0):
    fig, axs = plt.subplots(len(samples)*2,past+len(predictions_dict.keys()), figsize=(32, 32))
    fig.subplots_adjust(wspace=0.3, hspace=0.0)
    for n in range(len(samples)):
        vmax = np.max(test[n,:,:,:past])
        vmin = 0
        print(test.shape)
        for i in range(past):
            im = axs[2*n,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Past frame {i+1}")
            src.colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Past frame {i+1}")
            src.colorbar(im)
        for i in range(past,past+len(predictions_dict.keys())):
            im = axs[2*n,i].imshow(predictions_dict[f"{i-past}"][samples[n], :,:,0], vmax=vmax, vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Predicted frame {i-past+1}")
            src.colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax, vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Reference frame {i-past+1}")
            src.colorbar(im)
    fig.savefig(f"Plots/{name}_sequence_prediction.png")
    plt.close()


In [ ]:
save_examples(name, sequence_test, predictions, past, samples=[33])

Renormalize intensity values

In [ ]:
norms = np.load(sys.path[0]+"/5min_norms_compressed.npz")["arr_1"]

In [ ]:
# *4 bc of augmentaion (it concats the frames so the 0th 1000th 2000th and 3000th are the same sample just rotated)
test_norms = list(norms[9000:])*4

In [ ]:
#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in ['0', '1', '2', '3']])[:,:,:,:,0]), (1,2,3,0))

Calculate pixel intensities back to dBZ and from there to mm/h. <br>
Sources: <br>
- https://www.dwd.de/DE/leistungen/radolan/radolan_info/radolan_radvor_op_komposit_format_pdf.pdf?__blob=publicationFile&v=11 (page 10)
- <https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates>

In [ ]:
#dBZ
dBZ_t = renormalized_test*0.5 - 32.5
dBZ_p = renormalized_predictions*0.5 - 32.5
#mm/h
I_t = (0.005*10**(0.1*dBZ_t))**(0.625)
I_p = (0.005*10**(0.1*dBZ_p))**(0.625)

In [ ]:
#thresholds: 2, 8, 42
thresholds = [0.5]
scores = {}
for t in range(I_t.shape[-1]): # loop over the predictions (4)
    for s in thresholds: # make a dict entry for each threshold score
        scores[f"pred_{t+1}"] = src.calculate_skill_scores(I_p[...,t:t+1],
                                                          I_t[...,past+t:past+1+t],
                                                          x=I_t[...,past-1:past],
                                                          threshold=s)

In [ ]:
scores["pred_1"].keys()

In [ ]:
np.save(name+"_scores",scores)

In [ ]:
name

In [ ]:
loaded_scores = np.load(sys.path[1]+"/"+name"/"+name+"_scores.npy").item()

In [ ]:
list(loaded_scores.keys())

In [ ]:
np.mean((pd.Series(scores["pred_2"]["corr_to_truth"]).dropna()))

In [ ]:
np.mean((pd.Series(scores["pred_2"]["corr_to_input"]).dropna()))

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_name = 'block5_conv3'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
K.mean(layer_output[:, :, :, filter_index])